# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt','wordnet'])

from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import re
import pickle

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///ETL_prepration.db')
df = pd.read_sql_table('ETL_prepration',engine)

In [4]:
df.describe()

,id,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)","('search_and_rescue',)","('security',)","('military',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
X = df['message']
y = df[df.columns[4:]]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    #replace url with url place holder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    #extract word tokens form text
    tokens = word_tokenize(text)
    
    #Lemmanitizer to remove related forms of a word
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer= tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
y_pred = pipeline.predict(X_test)
for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i].values, y_pred[:,i], target_names=category_names.values))

('related',) :
                             precision    recall  f1-score   support

               ('related',)       0.61      0.32      0.42      1523
               ('request',)       0.81      0.94      0.87      4991
                 ('offer',)       0.50      0.15      0.23        40

                avg / total       0.76      0.79      0.76      6554

('request',) :
                             precision    recall  f1-score   support

               ('related',)       0.88      0.98      0.93      5404
               ('request',)       0.82      0.39      0.53      1150

                avg / total       0.87      0.88      0.86      6554

('offer',) :
                             precision    recall  f1-score   support

               ('related',)       1.00      1.00      1.00      6529
               ('request',)       0.00      0.00      0.00        25

                avg / total       0.99      1.00      0.99      6554

('aid_related',) :
                             pre

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [22]:
parameters = {'clf__estimator__min_samples_split': [2, 3, 4],
              'clf__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)

In [23]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 4.1min
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 4.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 4.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=40 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 7.4min
[CV] clf__estimator__min_samples_split=2, clf__estimato

[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed: 58.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__min_samples_split': [2, 3, 4], 'clf__estimator__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [24]:
#To find Best Params based on GridSearch
print(cv.best_params_)

{'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 40}


In [25]:
#New Model
Model_op = cv.best_estimator_
print(cv.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
y_pred = Model_op.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

('related',) :
             precision    recall  f1-score   support

          0       0.75      0.25      0.37      1523
          1       0.80      0.97      0.88      4991
          2       0.67      0.10      0.17        40

avg / total       0.79      0.80      0.76      6554

('request',) :
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5404
          1       0.87      0.44      0.59      1150

avg / total       0.89      0.89      0.88      6554

('offer',) :
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

('aid_related',) :
             precision    recall  f1-score   support

          0       0.77      0.86      0.82      3836
          1       0.77      0.64      0.70      2718

avg / total       0.77      0.77      0.77      6554

('medical_help',) :
     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [28]:
with open('Mlclassifier.pkl', 'wb') as file:
    pickle.dump(Model_op, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.